# Inserting the **DATA**

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import missingno as msno
from sklearn.preprocessing import MinMaxScaler
from imblearn.over_sampling import SMOTE
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.ensemble import GradientBoostingClassifier, RandomForestClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

In [ ]:
from google.colab import files
uploaded = files.upload()

Saving customer_churn_dataset-testing-master.csv to customer_churn_dataset-testing-master.csv
Saving customer_churn_dataset-training-master.csv to customer_churn_dataset-training-master.csv


In [ ]:
testDF = pd.read_csv('customer_churn_dataset-testing-master.csv').copy()
trainDF = pd.read_csv('customer_churn_dataset-training-master.csv').copy()

In [ ]:
testDF.drop(columns = ['CustomerID'])
testDF.drop(columns = ['CustomerID'])

,Age,Gender,Tenure,Usage Frequency,Support Calls,Payment Delay,Subscription Type,Contract Length,Total Spend,Last Interaction,Churn
0,22,Female,25,14,4,27,Basic,Monthly,598,9,1
1,41,Female,28,28,7,13,Standard,Monthly,584,20,0
2,47,Male,27,10,2,29,Premium,Annual,757,21,0
3,35,Male,9,12,5,17,Premium,Quarterly,232,18,0
4,53,Female,58,24,9,2,Standard,Annual,533,18,0
...,...,...,...,...,...,...,...,...,...,...,...
64369,45,Female,33,12,6,21,Basic,Quarterly,947,14,1
64370,37,Male,6,1,5,22,Standard,Annual,923,9,1
64371,25,Male,39,14,8,30,Premium,Monthly,327,20,1
64372,50,Female,18,19,7,22,Standard,Monthly,540,13,1


In [ ]:
trainDF['Total Spend / Age'] = trainDF['Total Spend'] / trainDF['Age']
trainDF['Age / Usage Frequency'] = trainDF['Age'] / trainDF['Usage Frequency']
trainDF['Total Spend / Usage Frequency'] = trainDF['Total Spend'] / trainDF['Usage Frequency']
trainDF['Support Calls/ Age']  = trainDF['Support Calls'] / trainDF['Age']

testDF['Total Spend / Age'] = testDF['Total Spend'] / testDF['Age']
testDF['Age / Usage Frequency'] = testDF['Age'] / testDF['Usage Frequency']
testDF['Total Spend / Usage Frequency'] = testDF['Total Spend'] / testDF['Usage Frequency']
testDF['Support Calls/ Age']  = testDF['Support Calls'] / testDF['Age']



In [ ]:
testDF.dropna(inplace=True)
trainDF.dropna(inplace=True)

In [ ]:
trainDF.isna().sum()

,0
CustomerID,0
Age,0
Gender,0
Tenure,0
Usage Frequency,0
Support Calls,0
Payment Delay,0
Subscription Type,0
Contract Length,0
Total Spend,0


In [ ]:
gender_numeric = {'Male': 0, 'Female': 1}

trainDF['Gender'] = trainDF['Gender'].map(gender_numeric)
testDF['Gender'] = testDF['Gender'].map(gender_numeric)
subtype_numeric = {'Basic' : 1, 'Standard' : 2, 'Premium' : 3}

trainDF['Subscription Type'] = trainDF['Subscription Type'].map(subtype_numeric)
testDF['Subscription Type'] = testDF['Subscription Type'].map(subtype_numeric)
contract_length_numeric = {'Monthly' : 1, 'Quarterly' : 2, 'Annual' : 3}

trainDF['Contract Length'] = trainDF['Contract Length'].map(contract_length_numeric)
testDF['Contract Length'] = testDF['Contract Length'].map(contract_length_numeric)

In [ ]:
trainDF.columns

Index(['CustomerID', 'Age', 'Gender', 'Tenure', 'Usage Frequency',
       'Support Calls', 'Payment Delay', 'Subscription Type',
       'Contract Length', 'Total Spend', 'Last Interaction', 'Churn',
       'Total Spend / Age', 'Age / Usage Frequency',
       'Total Spend / Usage Frequency', 'Support Calls/ Age'],
      dtype='object')

In [ ]:

print(testDF.shape[0] / trainDF.shape[0] * 100)


14.602841898954704


The top features are: ['Total Spend', 'Total Spend / Age',
       'Age / Usage Frequency', 'Total Spend / Usage Frequency',
       'Support Calls/ Age']. Thus, we will only maintain these features within the training, cross-validation, and test sets.

In [ ]:
X_train, X_val, y_train, y_val = train_test_split(trainDF.drop(columns=['Churn','CustomerID','Age','Gender','Tenure','Usage Frequency','Subscription Type','Last Interaction']), trainDF['Churn'], test_size=0.15, random_state=42)

In [ ]:
X_test = testDF.drop(columns=['Churn','CustomerID','Age','Gender','Tenure','Usage Frequency','Subscription Type','Last Interaction'])
y_test = testDF['Churn']

In [ ]:
best_models = []
metrics = []

metric_names = ['Accuracy', 'Precision', 'Recall', 'F1']
#model_names = ['Logistic Regression',  'Decision Tree Classifier', 'Gradient Boosting Classifier', 'Gaussian Naive Bayes']


In [ ]:
grid_search = GridSearchCV(LogisticRegression(), {'solver': ['newton-cg', 'lbfgs', 'liblinear'], 'penalty': ['l2'], 'C': [100, 10, 1.0, 0.1, 0.01]}, cv=5)
grid_search.fit(X_train, y_train.values.ravel())
best_model = grid_search.best_estimator_
best_models.append(best_model)

pred = best_model.predict(X_val)
accuracy = accuracy_score(y_val, pred)
precision = precision_score(y_val, pred)
recall = recall_score(y_val, pred)
f1 = f1_score(y_val, pred)

metrics.append({'Accuracy': accuracy, 'Precision': precision, 'Recall': recall, 'F1': f1})

print(f"Logisitic Regression Performance:")
for metric_name in metric_names:
  print(f"{metric_name}: {metrics[-1][metric_name]}")
  print()

/usr/local/lib/python3.10/dist-packages/sklearn/utils/optimize.py:44: LineSearchWarning: The line search algorithm did not converge
  ret = line_search_wolfe2(
/usr/local/lib/python3.10/dist-packages/sklearn/utils/optimize.py:204: UserWarning: Line Search failed
  warnings.warn("Line Search failed")
/usr/local/lib/python3.10/dist-packages/scipy/optimize/_linesearch.py:313: LineSearchWarning: Rounding errors prevent the line search from converging
  alpha_star, phi_star, old_fval, derphi_star = scalar_search_wolfe2(
/usr/local/lib/python3.10/dist-packages/sklearn/utils/optimize.py:44: LineSearchWarning: The line search algorithm did not converge
  ret = line_search_wolfe2(
/usr/local/lib/python3.10/dist-packages/sklearn/utils/optimize.py:204: UserWarning: Line Search failed
  warnings.warn("Line Search failed")
/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:460: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMI

Logisitic Regression Performance:
Accuracy: 0.8608695652173913

Precision: 0.8903400689995071

Recall: 0.8622454391175223

F1: 0.8760675701161194



In [ ]:
grid_search = GridSearchCV(DecisionTreeClassifier(), {'criterion': ['gini', 'entropy'], 'max_depth': [None, 5, 10, 20], 'min_samples_split': [2, 5, 10]}, cv=5)
grid_search.fit(X_train, y_train.values.ravel())
best_model = grid_search.best_estimator_
best_models.append(best_model)

pred = best_model.predict(X_val)
accuracy = accuracy_score(y_val, pred)
precision = precision_score(y_val, pred)
recall = recall_score(y_val, pred)
f1 = f1_score(y_val, pred)

metrics.append({'Accuracy': accuracy, 'Precision': precision, 'Recall': recall, 'F1': f1})

print(f"Decision Tree Classifier Performance:")
for metric_name in metric_names:
  print(f"{metric_name}: {metrics[-1][metric_name]}")
  print()

Decision Tree Classifier Performance:
Accuracy: 0.9865103969754253

Precision: 0.9984297162659735

Recall: 0.9778850233347476

F1: 0.9880505840745902



In [ ]:
grid_search = GridSearchCV(KNeighborsClassifier(), {'n_neighbors': [3, 5, 7], 'weights': ['uniform', 'distance']}, cv=5)
grid_search.fit(X_train, y_train.values.ravel())
best_model = grid_search.best_estimator_
best_models.append(best_model)

pred = best_model.predict(X_val)
accuracy = accuracy_score(y_val, pred)
precision = precision_score(y_val, pred)
recall = recall_score(y_val, pred)
f1 = f1_score(y_val, pred)

metrics.append({'Accuracy': accuracy, 'Precision': precision, 'Recall': recall, 'F1': f1})

print(f"K Neighbors Classifier Performance:")
for metric_name in metric_names:
  print(f"{metric_name}: {metrics[-1][metric_name]}")
  print()

K Neighbors Classifier Performance:
Accuracy: 0.9062230623818526

Precision: 0.9833123715451394

Recall: 0.8499946966482818

F1: 0.9118061185305286



In [ ]:
grid_search = GridSearchCV(GradientBoostingClassifier(), {'n_estimators': [50, 100, 200], 'learning_rate': [0.1, 0.05, 0.01]}, cv=5)
grid_search.fit(X_train, y_train.values.ravel())
best_model = grid_search.best_estimator_
best_models.append(best_model)

pred = best_model.predict(X_val)
accuracy = accuracy_score(y_val, pred)
precision = precision_score(y_val, pred)
recall = recall_score(y_val, pred)
f1 = f1_score(y_val, pred)

metrics.append({'Accuracy': accuracy, 'Precision': precision, 'Recall': recall, 'F1': f1})

print(f"Gradient Boosting Classifier Performance:")
for metric_name in metric_names:
  print(f"{metric_name}: {metrics[-1][metric_name]}")
  print()

Gradient Boosting Classifier Performance:
Accuracy: 0.9862986767485822

Precision: 0.9998641894828335

Recall: 0.9761084005091217

F1: 0.9878434950622583



In [ ]:
from xgboost import XGBClassifier


In [ ]:
grid_search = GridSearchCV(XGBClassifier(), {}, cv=5)
grid_search.fit(X_train, y_train.values.ravel())
best_model = grid_search.best_estimator_
best_models.append(best_model)

pred = best_model.predict(X_val)
accuracy = accuracy_score(y_val, pred)
precision = precision_score(y_val, pred)
recall = recall_score(y_val, pred)
f1 = f1_score(y_val, pred)

metrics.append({'Accuracy': accuracy, 'Precision': precision, 'Recall': recall, 'F1': f1})

print(f"XG Boost Classifier Performance:")
for metric_name in metric_names:
  print(f"{metric_name}: {metrics[-1][metric_name]}")
  print()

In [ ]:
metrics_df = pd.DataFrame(metrics, index=[f"Model {i+1}" for i in range(len(metrics))])


In [ ]:
print(metrics)

In [ ]:
plt.figure(figsize=(10, 6))
metrics_df.plot(kind='bar')
plt.xlabel('Model')
plt.ylabel('Metric Value')
plt.title('Comparison of Classification Metrics')
plt.legend(loc='lower right')
plt.show()

Since XG Boost resulted in the best performance, XG Boost will be the final training model and be tested against the Customer Churn.